# Scikit-Learn Cross Val Score on Logistic Regression Model
## Using IRIS_VIEW from DWC. This view has 150 records.

## Install fedml-azure package

In [1]:
pip install fedml-azure --force-reinstall

Processing ./fedml_azure-1.0.0-py3-none-any.whl
  Using cached hdbcli-2.10.13-cp34-abi3-manylinux1_x86_64.whl (11.7 MB)
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.10.13
    Uninstalling hdbcli-2.10.13:
      Successfully uninstalled hdbcli-2.10.13
  Attempting uninstall: fedml-azure
    Found existing installation: fedml-azure 1.0.0
    Uninstalling fedml-azure-1.0.0:
      Successfully uninstalled fedml-azure-1.0.0
Note: you may need to restart the kernel to use updated packages.


## Import the libraries needed in this notebook

In [2]:
from fedml_azure import create_workspace
from fedml_azure import create_compute
from fedml_azure import create_environment
from fedml_azure import DwcAzureTrain

## Set up
### Creating a workspace. This takes a dictionary as input for parameter workspace_args.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, 
resource_group, and workspace_name with your information.
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python


In [3]:
#creation of workspace
workspace=create_workspace(workspace_args={
                                            "subscription_id": '<subscription-id>',
                                            "resource_group": '<resource-group>',
                                            "workspace_name": '<workspace_name>'
                                            }
)



Getting existing Workspace


### Creating a Compute Cluster. This takes the workspace, a compute_type, and compute_args.
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

In [4]:
#creation of compute target
compute=create_compute(workspace=workspace,
                   compute_type='AmlComputeCluster',
                   compute_args={'vm_size':'Standard_D12_v2',
                                'vm_priority':'lowpriority',
                                'compute_name':'cpu-clu-cross',
                                'min_nodes':0,
                                'max_nodes':4,
                                'idle_seconds_before_scaledown':1700
                                }
                )

Creating Compute_target
Found compute target. just use it. cpu-clu-cross


### Creating an Environment. This takes the workspace, environment_type, and environment_args.

The fedml-azure pip package must be passed to the pip_packages key in the environment_args and 
to use scikit-learn, you must pass the name to conda_packages as well.

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments

In [5]:
#creation of environment
environment=create_environment(workspace=workspace, 
                            environment_type='CondaPackageEnvironment',
                           environment_args={'name':'test-env-cross',
                                            'conda_packages':['scikit-learn'],
                                            'pip_packages':['fedml-azure']})

Creating Environment


## Now, let's train the model
### First, we need to instantiate the training class - this will assign the workspace, compute, and environment

In [6]:
train=DwcAzureTrain(workspace=workspace, experiment_args={'name':'test-2'}, compute=compute, environment=environment)

Assigning Workspace
Creating Experiment
Assigning compute
Assigning Environment


## Here we are updating our compute from a compute cluster to a compute instance. This is optional to run.

In [7]:
train.update_compute(compute_type='AmlComputeInstance',compute_args={'vm_size':"Standard_D3_v2",'compute_name':'cpu-fedml'})

Updating compute
Creating Compute_target

Creating........................................
Running


### Then, we need to generate the run config. This is needed to package the configuration specified so we can submit a job for training. 


Before running the following cell, you should have a config.json file with the specified values to allow you to access to DWC. Provide this file path to config_file_path in the below cell.

You should also have the follow view IRIS_VIEW created in your DWC. To gather this data, please refer to https://www.kaggle.com/uciml/iris

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

In [8]:
#generating the run config
src=train.generate_run_config(config_file_path='dwc_configs/config.json',
                          config_args={
                                          'source_directory':'Scikit-Learn-CrossValScore',
                                          'script':'iris_train.py',
                                          'arguments':[
                                                        '--model_file_name','regression.pkl',
                                                        '--table_name', 'IRIS_VIEW',
                                                        '--table_size', '1']
                                          }
                            )

Generating script run config
Config file already exists in the script_directory Scikit-Learn-CrossValScore


### Submitting the job for training

In [9]:
#submitting the training run
run=train.submit_run(src)

Submitting training run
RunId: test-2_1633631577_4a4fbb73
Web View: https://ml.azure.com/runs/test-2_1633631577_4a4fbb73?wsid=/subscriptions/cb97564e-cea8-45a4-9c5c-a3357e8f7ee4/resourcegroups/Sample2_AzureML_Resource/workspaces/Sample2_AzureML_Worskpace&tid=42f7676c-f455-423c-82f6-dc2d99791af7

Streaming azureml-logs/55_azureml-execution-tvmps_1afe99b5a2cc83b393f3735fe9660c4edc4b55114e22a21f9a1eda2a5559fd9e_d.txt

2021-10-07T18:33:04Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/sample2_azureml_worskpace/azureml/test-2_1633631577_4a4fbb73/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/sample2_azureml_worskpace/azureml/test-2_1633631577_4a4fbb73/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=171371 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/sample2_azureml_worskpace/azureml/test-2_1633631577_4a4fbb73/configs/workspaceblobstore.cfg --log-le

## Register the model for deployment

In [10]:
# register the best model with the input dataset
model = train.register_model(run=run,
                           model_args={'model_name':'sklearn_crossval_model', 
                           'model_path':'outputs/regression.pkl'},
                           resource_config_args={'cpu':1, 'memory_in_gb':0.5},
                           is_sklearn_model=True
                            )

print('Name:', model.name)
print('Version:', model.version)

Registering the model
Configuring parameters for sklearn model
Name: sklearn_crossval_model
Version: 6
